In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [25]:
db = client.recipe_db
collection = db.low_calorie

In [26]:
url = 'https://www.allrecipes.com/recipes/84/healthy-recipes/'

response = requests.get(url)

soup = bs(response.text, 'html.parser')

In [27]:
results = soup.find_all('a', class_='carouselNav__link recipeCarousel__link')
print(results)

recipes/low-calorie/" tabindex="0">
<div alt="Low-Calorie Recipes" aria-label="" class="component lazy-image lazy-image-udf aspect_1x1 align-default" data-alt="Low-Calorie Recipes" data-content-type="" data-crop-percentage="100" data-high-density="false" data-main-recipe="" data-orientation="default" data-original-height="960" data-original-width="960" data-shop-image="false" data-special-crop="" data-src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2F2020%2F07%2F22%2F1986310.jpg" data-title="Low-Calorie Recipes" data-tracking-zone="image" data-width="      100
                                   ">
<noscript>
<img alt="Low-Calorie Recipes" src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2F2020%2F07%2F22%2F1986310.jpg"/>
</noscript>
<div class="inner-container js-inner-container">
</div>
<div class="lazy-image__loadingPlaceholder">
<img alt="" 

In [28]:
health_cats = ['low-calorie recipes','low-carb recipes','low-cholesterol recipes','low-fat recipes', 'weight-loss recipes', 'healthy snacks']


In [29]:
cat = [x['href'] for x in results if x['data-tracking-content-headline'] in health_cats]

In [33]:
main_cats={}

for c in cat:
    r = requests.get(c)
    soup = bs(r.text, 'html.parser')
    title = soup.find('h1', class_='category-page-heading').text
    results = soup.find_all('a', class_= 'card__titleLink manual-link-behavior')
    links = [y['href'] for y in results]
    main_cats[title] = list(set(links))
    
main_cats

{'Healthy Snack Recipes': ['https://www.allrecipes.com/recipe/223467/granola-bars/',
  'https://www.allrecipes.com/recipe/246372/perfectly-dry-roasted-chickpeas/',
  'https://www.allrecipes.com/recipe/222082/healthier-banana-banana-bread/',
  'https://www.allrecipes.com/recipe/260520/apple-frog-for-kids/',
  'https://www.allrecipes.com/recipe/176957/baked-kale-chips/',
  'https://www.allrecipes.com/recipe/81298/playgroup-granola-bars/',
  'https://www.allrecipes.com/recipe/26692/annies-fruit-salsa-and-cinnamon-chips/',
  'https://www.allrecipes.com/recipe/54346/applesauce/',
  'https://www.allrecipes.com/recipe/254364/baked-sweet-potato-coins/',
  'https://www.allrecipes.com/recipe/236557/salt-and-vinegar-potato-chips/',
  'https://www.allrecipes.com/recipe/54165/balsamic-bruschetta/',
  'https://www.allrecipes.com/recipe/236025/salt-and-pepper-skillet-fries/',
  'https://www.allrecipes.com/recipe/16259/ds-famous-salsa/',
  'https://www.allrecipes.com/recipe/87305/microwave-popcorn/',


In [44]:
main_cats['Healthy Snack Recipes'][0]

'https://www.allrecipes.com/recipe/223467/granola-bars/'

In [49]:
for k, v in main_cats.items():
    collection = db.main_cats[k]
    for s in v:
        print(s)
        resp = requests.get(s)
        soup = bs(resp.text, 'html.parser')
        title = soup.find('h1', class_= 'headline heading-content').text
        # Add a try / except arg for rating and num_ratings
        rating = soup.find('span', class_= 'review-star-text').text.replace('\n','').strip()
        num_ratings = soup.find('span', class_= 'ugc-ratings-item').text.replace('\n','').strip()
        time = soup.find_all('div', class_= 'recipe-meta-item')
        time_dict={}
        time_dict['prep'] = time[0].find('div', class_= 'recipe-meta-item-body').text.replace('\n','').strip()
        time_dict['total'] = time[1].find('div', class_= 'recipe-meta-item-body').text.replace('\n','').strip()
        ingredients = soup.find_all('span', class_= 'ingredients-item-name')
        ingredient_list = [ingredient.text.replace('\n','').strip() for ingredient in ingredients]
        nutrients = soup.find_all('span', class_= 'nutrient-name')
        nutrient_dict={}
        for nutrient in nutrients:
            name = nutrient.text.replace('\n',"").strip().split(':')
            nutrient_dict[name[0]] = name[1].strip()
        
        print('--------')
        print(k)
        print(s)
        print(title)
        print(rating)
        print(num_ratings)
        print(time_dict)
        print(ingredient_list)
        print(nutrient_dict)


https://www.allrecipes.com/recipe/223467/granola-bars/
--------
Healthy Snack Recipes
https://www.allrecipes.com/recipe/223467/granola-bars/
Granola Bars
Rating: 4.35 stars
107 Ratings
{'prep': '15 mins', 'total': '25 mins'}
['2 cups rolled oats', '½ cup shredded coconut', '½ cup honey', '2 tablespoons creamy peanut butter', '1 teaspoon vanilla extract', '⅛ teaspoon salt']
{'protein': '3.9g', 'carbohydrates': '34.4g', 'dietary fiber': '2.8g', 'sugars': '19.7g', 'fat': '4.7g', 'saturated fat': '1.9g', 'niacin equivalents': '1.6mg', 'vitamin c': '0.1mg', 'folate': '10.1mcg', 'calcium': '14.1mg', 'iron': '1.1mg', 'magnesium': '37.1mg', 'potassium': '128.2mg', 'sodium': '70.3mg', 'thiamin': '0.2mg', 'calories from fat': '42'}
https://www.allrecipes.com/recipe/246372/perfectly-dry-roasted-chickpeas/
--------
Healthy Snack Recipes
https://www.allrecipes.com/recipe/246372/perfectly-dry-roasted-chickpeas/
Perfectly Dry Roasted Chickpeas
Rating: 4.09 stars
11 Ratings
{'prep': '5 mins', 'total':

AttributeError: 'NoneType' object has no attribute 'text'